In [ ]:
######JM Gawrych  11/30/2018 rev 3/11/2019
#### Gets multiple Hail Vison geojson files from our featurizer catalog and combines them

import requests,json,datetime # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(335) # Radar Derived 24-Hour HailVision. 
prod_name='MESHMax1440min' # Per TWC naming conventions
APIKEY='THE_WEATHER_COMPANY_DATA_PACKAGE_KEY_HERE'  #Enter your API key 


########################PRODUCT INVENTORY CALL################################  
def InvCall():
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2
    print("Call #1: " + url)
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json() 
        for x in range(0,1):
            for y in range(0,1):
                timestamp = data["layers"][prod_num][prod_name]["dimensions"][x]["t"][y]
            return timestamp

##########################   API CALL   ###################################
def APIcall(timestamp,ts): # function for pulling data 
    count=0

    num_layers = 6  #10
    inc=12.7  
    fc = {
        "type": "FeatureCollection",
        "features": []
    }
   
    low_limit = 12.7   ##Thresholds are in mm, min is 5 mm (0.2") up to 50 mm (~2") 
    low_limitS = str(low_limit)
    high_limit=low_limit+12.7
    high_limitS = str(high_limit)      
                 
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&t='+timestamp + '&apiKey='+APIKEY+'&geometryType=polygon'
    
    for x in range(0,num_layers):
        
        count=count+1
        base3 = '&threshold='+low_limitS

        url = base1 + base2 + base3  # make API URL   
        print ("Call #2: " + url)
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']
            #convert to inches
            HailSizeIn= round(float(y['properties']['threshold'])*0.0394,1)
            f['properties']['HailSize'] = HailSizeIn  # y['properties']['threshold']
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
          
        low_limit=low_limit+inc
        low_limitS=str(low_limit)                  
        high_limit=high_limit+inc
        high_limitS=str(high_limit)   
           
    with open("HailVision.geojson", "w") as outfile:
        json.dump(fc, outfile)
    ################################################################################
    ##  ADD CONTENT TO ARCGIS  
    # Connect to the GIS Using Your Credentials
    portalURL = "http://ibmmain.maps.arcgis.com"
    userID = "ESRI_USERNAME"
    password = "ESRI_PASSWORD"
    gis = GIS(portalURL, userID, password)
    #Update each piece of content
    existing_ID = '465ef129d4314343864b73f49d48c83e'  # Map ID
    wx_item = gis.content.get(existing_ID)
    collection = features.FeatureLayerCollection.fromitem(wx_item)
    collection.manager.overwrite('HailVision.geojson')  
    print ("Updating Layer")


################################################################################
timestamp = InvCall()  ## Grab time stamp
print(timestamp)
fmt = "%Y-%m-%d %H:%M:%S"
# local time
#t = datetime.datetime.fromtimestamp(int(timeInfo[1])/1000)
#print (t.strftime(fmt)) # prints 2012-08-28 02:45:17
# utc time
t_utc = datetime.datetime.utcfromtimestamp(int(timestamp)/1000)
t_utc_formatted = t_utc.strftime(fmt)
APIcall(timestamp,t_utc_formatted)